In [1]:
# Dependencies
import pandas as pd
import requests
import json
from db_key import mysql_user, mysql_password
from pprint import pprint


In [2]:
# Setting parameters which will be used to obtain data from fda.gov

years = ['2013','2014','2015','2016','2017','2018','2019']
date_begin = ['0101','0301','0501','0701','0901','1101']
dates_end = ['0228','0430','0630','0831','1031','1231']
iteration = 1
frames = []

# Looping through lists and receiving data over API requests
# Added error handling section in case if there will be no data
for year in years:
    for  x, y in zip(date_begin, dates_end):
        
        url = "https://api.fda.gov/food/enforcement.json?search=report_date:["+year+x+"+TO+"+year+y+"]&limit=99"
        print ("Requesting data for time period: " + year + "-" + x + "+TO+"+ year + "-" + y)
        respond = requests.get(url).json()
        try:
            df_temp = pd.DataFrame(respond["results"])
            df_clean = df_temp[['state', 'city', 'report_date', 'product_description', 'product_quantity', 'reason_for_recall', 'recalling_firm', 'recall_number']]
            frames.append(df_clean)
        except:
            print('NULL RESPOND')

        iteration = iteration + 1
        
        if iteration > 40:
            break
            
result = pd.concat(frames)

Requesting data for time period: 2013-0101+TO+2013-0228
Requesting data for time period: 2013-0301+TO+2013-0430
Requesting data for time period: 2013-0501+TO+2013-0630
Requesting data for time period: 2013-0701+TO+2013-0831
Requesting data for time period: 2013-0901+TO+2013-1031
Requesting data for time period: 2013-1101+TO+2013-1231
Requesting data for time period: 2014-0101+TO+2014-0228
Requesting data for time period: 2014-0301+TO+2014-0430
Requesting data for time period: 2014-0501+TO+2014-0630
Requesting data for time period: 2014-0701+TO+2014-0831
Requesting data for time period: 2014-0901+TO+2014-1031
Requesting data for time period: 2014-1101+TO+2014-1231
Requesting data for time period: 2015-0101+TO+2015-0228
Requesting data for time period: 2015-0301+TO+2015-0430
Requesting data for time period: 2015-0501+TO+2015-0630
Requesting data for time period: 2015-0701+TO+2015-0831
Requesting data for time period: 2015-0901+TO+2015-1031
Requesting data for time period: 2015-1101+TO+20

In [3]:
# Sorting data by date, renaming columns

result.sort_values(by=['report_date'])
result = result.rename(columns={"recalling_firm":"brand"})
result['brand'] = result['brand'].str.lower()
result.head()

,state,city,report_date,product_description,product_quantity,reason_for_recall,brand,recall_number
0,TX,Austin,20130116,Kentucky Bleu cheese was sold in the Louisvill...,1.21 lbs,Whole Foods Market announces that it is recall...,"whole foods market brand 365, llc",F-0868-2013
1,FL,Doral,20130130,"Cold Smoke Salmon in vacuum plastic bags, labe...","1,080 lbs (1,436 lbs on hold)",Potential to be contaminated with Listeria Mon...,"marine harvest usa, llc",F-0930-2013
2,CA,El Monte,20130109,Sanh Yuan Preserved Plum Candy (Sweet and Sour...,430 packs,CHO Fuku Group Inc. is recalling Sanh Yuan Pre...,cho fuku group (usa) inc,F-0855-2013
3,CA,Sausalito,20130123,Somersault brand Cinnamon Crunch. Crunchy nug...,"36,757 cases",During ingredient review for kosher certificat...,"somersault snack co., llc",F-0879-2013
4,OH,Sidney,20130123,"Holiday Fruit Delight Kit 5.1875 lbs (2,353g) ...",6 cs/5.1875 lb bags,Possible contamination with Listeria monocytog...,fresh unlimited inc,F-0896-2013


In [4]:
# Creating new DataFrame with brand names to remove redundancy

brands = list(result.brand.unique())
brands_df = pd.DataFrame(brands, columns=['brand'])
brands_df.head()

,brand
0,"whole foods market brand 365, llc"
1,"marine harvest usa, llc"
2,cho fuku group (usa) inc
3,"somersault snack co., llc"
4,fresh unlimited inc


In [5]:
# loading large tsv file (963 MB)
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Data/en.openfoodfacts.org.products.tsv', sep='\t', header=0)

In [6]:
# Keeping only some columns, cleaning data

df = df[['product_name', 'brands','ingredients_text', 'countries']]
df = df.loc[df['countries'] == 'US', :]
df = df[['product_name', 'brands','ingredients_text']]
df['brands'] = df['brands'].str.lower()

df = df.dropna()
df = df.reset_index(drop=True)
brands = list(df.brands.unique())
brands_df_new = pd.DataFrame(brands, columns=['brand'])


In [7]:
# Setting same name for brand column in both DataFrames, creating full brand list
df = df.rename(columns={"brands":"brand"})
total_brand_list = pd.concat([brands_df, brands_df_new])

In [8]:
# New brand list with only unique names
total_brand_list.nunique()
brands = list(total_brand_list.brand.unique())

In [9]:
# Doing the same for another DF
id_brand_table = pd.DataFrame(brands, columns=['brand'])
id_brand_table.head()

,brand
0,"whole foods market brand 365, llc"
1,"marine harvest usa, llc"
2,cho fuku group (usa) inc
3,"somersault snack co., llc"
4,fresh unlimited inc


In [10]:
id_brand_table = id_brand_table.reset_index()
id_brand_table = id_brand_table.rename(columns={"index":"brand_id"})
id_brand_table.head()

,brand_id,brand
0,0,"whole foods market brand 365, llc"
1,1,"marine harvest usa, llc"
2,2,cho fuku group (usa) inc
3,3,"somersault snack co., llc"
4,4,fresh unlimited inc


In [11]:
# Mergind brand table with initial table and removing names so we get only index of brand name in this table

new_df = pd.merge(id_brand_table, result, on='brand', how='outer')
new_df = new_df[['brand_id', 'state', 'city', 'report_date', 'product_description', 'product_quantity', 'reason_for_recall', 'recall_number']]
new_df = new_df.dropna()
new_df.head()


,brand_id,state,city,report_date,product_description,product_quantity,reason_for_recall,recall_number
0,0,TX,Austin,20130116,Kentucky Bleu cheese was sold in the Louisvill...,1.21 lbs,Whole Foods Market announces that it is recall...,F-0868-2013
1,0,TX,Austin,20170222,"Whole Foods Market, TART LEMON SMALL",,"According to the firm, packaged crusts used in...",F-1477-2017
2,0,TX,Austin,20170222,"Whole Foods Market, TART KEY LIME 4IN TART KE...",4822,"According to the firm, packaged crusts used in...",F-1472-2017
3,1,FL,Doral,20130130,"Cold Smoke Salmon in vacuum plastic bags, labe...","1,080 lbs (1,436 lbs on hold)",Potential to be contaminated with Listeria Mon...,F-0930-2013
4,1,FL,Doral,20130130,"Cold Smoke Salmon in vacuum plastic bags, labe...",,Potential to be contaminated with Listeria Mon...,F-0933-2013


In [12]:
# Performing same steps on other columns, now values will be replaced with index

city = pd.DataFrame(list(new_df.city.unique()),columns=['city'])
city = city.reset_index()
city = city.rename(columns={"index":"city_id"})
new_df_temp = pd.merge(new_df, city, on='city', how='outer')
city.drop(['city_id'], axis=1, inplace=True)

state = pd.DataFrame(list(new_df.state.unique()), columns=['state'])
state = state.reset_index()
state = state.rename(columns={"index":"state_id"})
new_df_temp = pd.merge(new_df_temp, state, on='state', how='outer')
state.drop(['state_id'], axis=1, inplace=True)

reason_for_recall = pd.DataFrame(list(new_df.reason_for_recall.unique()), columns=['reason_for_recall'])
reason_for_recall = reason_for_recall.reset_index()
reason_for_recall = reason_for_recall.rename(columns={"index":"reason_for_recall_id"})
new_df_temp = pd.merge(new_df_temp, reason_for_recall, on='reason_for_recall', how='outer')
reason_for_recall.drop(['reason_for_recall_id'], axis=1, inplace=True)

new_df_temp.drop(['state', 'city', 'reason_for_recall'], axis=1, inplace=True)
new_df = new_df_temp[['recall_number','product_quantity','product_description','report_date','brand_id','reason_for_recall_id','city_id','state_id']]
new_df.head()

,recall_number,product_quantity,product_description,report_date,brand_id,reason_for_recall_id,city_id,state_id
0,F-0868-2013,1.21 lbs,Kentucky Bleu cheese was sold in the Louisvill...,20130116,0,0,0,0
1,F-1477-2017,,"Whole Foods Market, TART LEMON SMALL",20170222,0,1,0,0
2,F-1472-2017,4822,"Whole Foods Market, TART KEY LIME 4IN TART KE...",20170222,0,1,0,0
3,F-1144-2013,56 cases (1120 cartons),"365 Organic Orzo, 16oz, 20 carton/case",20130320,83,133,0,0
4,F-1006-2013,"1,035 cases (12,420 retail units)",Whole Foods Market Panettone Chocolate - 26.4 ...,20130306,83,134,0,0


In [13]:
new_df_1 = pd.merge(id_brand_table, df, on='brand', how='outer')
new_df_1 = new_df_1[['brand_id', 'product_name', 'ingredients_text']]
new_df_1 = new_df_1.dropna()
id_brand_table = id_brand_table[['brand']]
id_brand_table.head()

,brand
0,"whole foods market brand 365, llc"
1,"marine harvest usa, llc"
2,cho fuku group (usa) inc
3,"somersault snack co., llc"
4,fresh unlimited inc


In [14]:
# Connecting to mySql and loading data into it

import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user=mysql_user,
  passwd=mysql_password)

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS fda_db")

In [15]:
from pandas.io import sql
from sqlalchemy import create_engine
engine = create_engine('mysql+pymysql://' + mysql_user + ':' + mysql_password + '@localhost/fda_db')
with engine.connect() as con, con.begin():
    id_brand_table.to_sql(con=con, name='id_brand', if_exists='replace')
    new_df.to_sql(con=con, name='fda_recalls', if_exists='replace')
    new_df_1.to_sql(con=con, name='open_food_facts', if_exists='replace')
    city.to_sql(con=con, name='city', if_exists='replace')
    state.to_sql(con=con, name='state', if_exists='replace')
    reason_for_recall.to_sql(con=con, name='reason_for_recall', if_exists='replace')